## MF기반 잠재요인 협업 필터링

In [62]:
import pandas as pd
import numpy as np

book = pd.read_csv('gyobo_crawling_all.csv')

In [63]:
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [64]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
    
    # P와 Q 매트릭스를 계속 업데이트(확률적 경사하강법)
    for step in tqdm_notebook(range(steps)):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
        
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [65]:
pwd

'C:\\Users\\insoo\\Desktop\\Untitled Folder\\crawler\\gyobo'

In [66]:
book

,Unnamed: 0,Unnamed: 0.1,id,klover,comment,keyword,score,total,title
0,0,0,st**05,4,추천받자마자 망설임없이 바로 구매한 책입니다.(덕분에 선착순으로 예쁜 초도 같이 받...,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,첫 번째 디자인 케이크
1,1,1,db**jd661,4,평이 좋아서 구매했는데 활용하기 좋은거 같아요,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,첫 번째 디자인 케이크
2,2,2,an**ldms89,4,완전좋아요 꼭사세요,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,첫 번째 디자인 케이크
3,3,3,ky**ng4757,4,읽기 편하고 좋아요 만족합니다,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,첫 번째 디자인 케이크
4,4,4,mi**2013,4,더 많은 디자인이 있으면 좋겠어요,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,첫 번째 디자인 케이크
...,...,...,...,...,...,...,...,...,...
182109,6788,177,hy**onsj,4,사실 좋대서 삼^^,전치사 공부 영어회화 손흥민 관사 플로리다,9.4,182,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드)
182110,6789,178,su**n2465,4,좋아요,전치사 공부 영어회화 손흥민 관사 플로리다,9.4,182,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드)
182111,6790,179,gp**1890,4,좋아요,전치사 공부 영어회화 손흥민 관사 플로리다,9.4,182,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드)
182112,6791,180,gh**dkr,4,입문단계에서 적절해요,전치사 공부 영어회화 손흥민 관사 플로리다,9.4,182,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드)


In [67]:
# MF알고리즘 사용하기 위해 필요한 컬럼 분리
ratings = book[['id', 'title', 'klover']]

In [68]:
ratings_matrix = ratings.pivot_table('klover', index='id', columns='title')
# 사용자-아이템 행렬 : R(원본 행렬)

In [69]:
# 필요한 컬럼만 추출
rating_book = book[['id', 'title', 'klover', "keyword"]]

### MF위한 데이터 전처리

In [70]:
# id 비어있는 행 찾기
rating_book.id.isnull().sum()

3317

In [71]:
# id 비어있는 행 삭제
rating_book = rating_book.dropna(axis=0)
rating_book.id.isnull().sum()

0

In [72]:
rating_book

,id,title,klover,keyword
0,st**05,첫 번째 디자인 케이크,4,디저트 초코 아이싱 플라워 제과제빵 밀크티
1,db**jd661,첫 번째 디자인 케이크,4,디저트 초코 아이싱 플라워 제과제빵 밀크티
2,an**ldms89,첫 번째 디자인 케이크,4,디저트 초코 아이싱 플라워 제과제빵 밀크티
3,ky**ng4757,첫 번째 디자인 케이크,4,디저트 초코 아이싱 플라워 제과제빵 밀크티
4,mi**2013,첫 번째 디자인 케이크,4,디저트 초코 아이싱 플라워 제과제빵 밀크티
...,...,...,...,...
182109,hy**onsj,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드),4,전치사 공부 영어회화 손흥민 관사 플로리다
182110,su**n2465,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드),4,전치사 공부 영어회화 손흥민 관사 플로리다
182111,gp**1890,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드),4,전치사 공부 영어회화 손흥민 관사 플로리다
182112,gh**dkr,미국 영어 회화 문법 1: 명사 관련 활용(MP3 다운로드),4,전치사 공부 영어회화 손흥민 관사 플로리다


In [73]:
# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_book.pivot_table('klover', index='id', columns='title')

In [74]:
# NaN값 확인
ratings_matrix

title,-10KG 밀가루 단식,0.1%의 비밀,0~5세 뇌가 쑥쑥 자라는 놀이 육아,0~5세 말걸기 육아의 힘,1% 학습 코칭(99% 헛고생을 피하는),100 인생 그림책(Dear 그림책)(양장본 HardCover),1000개 숨은그림찾기: 바다 동물,1000개 숨은그림찾기: 아이스크림(찾아도 찾아도 끝판왕),1000개의 그림 1000가지 공감,100층짜리 집(양장본 HardCover),...,흔한남매의 THE 재즈소곡집 Very Easy,흔한남매의 흔한 호기심. 1,흔한남매의 흔한 호기심. 2,흔한남매의 흔한 호기심. 3,흔한남매의 흔한 호기심. 4,흔한남매의 흔한 호기심. 5,희망의 봄,히데코의 사적인 안주 교실,힐링 코드(개정증보판),힐튼호텔 옆 쪽방촌 이야기
id,,,,,,,,,,,,,,,,,,,,,
00**3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**aha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**aith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**ang13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zz**zzumi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zz**zzz46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
나**나야,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# 알고리즘 동작 초과범위 넘지 않기 위해 행 50,000개로 변경
ratings_matrix2 = ratings_matrix[:50000]

In [76]:
print(ratings_matrix2.shape)
ratings_matrix2.head()

(50000, 3095)


title,-10KG 밀가루 단식,0.1%의 비밀,0~5세 뇌가 쑥쑥 자라는 놀이 육아,0~5세 말걸기 육아의 힘,1% 학습 코칭(99% 헛고생을 피하는),100 인생 그림책(Dear 그림책)(양장본 HardCover),1000개 숨은그림찾기: 바다 동물,1000개 숨은그림찾기: 아이스크림(찾아도 찾아도 끝판왕),1000개의 그림 1000가지 공감,100층짜리 집(양장본 HardCover),...,흔한남매의 THE 재즈소곡집 Very Easy,흔한남매의 흔한 호기심. 1,흔한남매의 흔한 호기심. 2,흔한남매의 흔한 호기심. 3,흔한남매의 흔한 호기심. 4,흔한남매의 흔한 호기심. 5,희망의 봄,히데코의 사적인 안주 교실,힐링 코드(개정증보판),힐튼호텔 옆 쪽방촌 이야기
id,,,,,,,,,,,,,,,,,,,,,
00**3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**aha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**aith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00**ang13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 실행

In [77]:
%%time
# 경사하강법을 이용한 행렬 분해(4~5분 정도 걸림)
P, Q = matrix_factorization(ratings_matrix2.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)

pred_matrix = np.dot(P, Q.T)

  0%|          | 0/200 [00:00<?, ?it/s]

### iteration step :  0  rmse :  3.876243618255687
### iteration step :  10  rmse :  1.0873313890999476
### iteration step :  20  rmse :  0.5570594241303363
### iteration step :  30  rmse :  0.3516129185306797
### iteration step :  40  rmse :  0.24401920712908684
### iteration step :  50  rmse :  0.1784909056803411
### iteration step :  60  rmse :  0.13597734339099612
### iteration step :  70  rmse :  0.10676240216819534
### iteration step :  80  rmse :  0.08562909957788893
### iteration step :  90  rmse :  0.07149409094218029
### iteration step :  100  rmse :  0.06290055328750249
### iteration step :  110  rmse :  0.05752463555671283
### iteration step :  120  rmse :  0.053833211249086306
### iteration step :  130  rmse :  0.051084894276952855
### iteration step :  140  rmse :  0.04891519495186326
### iteration step :  150  rmse :  0.047129152658404186
### iteration step :  160  rmse :  0.045614992238059024
### iteration step :  170  rmse :  0.04430408554964674
### iteration step :  1

In [78]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix2.index,
                                   columns = ratings_matrix2.columns)
# 예측 평점 행렬 확인
print(ratings_pred_matrix.shape)
ratings_pred_matrix.tail(100)

(50000, 3095)


title,-10KG 밀가루 단식,0.1%의 비밀,0~5세 뇌가 쑥쑥 자라는 놀이 육아,0~5세 말걸기 육아의 힘,1% 학습 코칭(99% 헛고생을 피하는),100 인생 그림책(Dear 그림책)(양장본 HardCover),1000개 숨은그림찾기: 바다 동물,1000개 숨은그림찾기: 아이스크림(찾아도 찾아도 끝판왕),1000개의 그림 1000가지 공감,100층짜리 집(양장본 HardCover),...,흔한남매의 THE 재즈소곡집 Very Easy,흔한남매의 흔한 호기심. 1,흔한남매의 흔한 호기심. 2,흔한남매의 흔한 호기심. 3,흔한남매의 흔한 호기심. 4,흔한남매의 흔한 호기심. 5,희망의 봄,히데코의 사적인 안주 교실,힐링 코드(개정증보판),힐튼호텔 옆 쪽방촌 이야기
id,,,,,,,,,,,,,,,,,,,,,
so**alcu8,2.788720,2.785159,2.995132,3.090255,2.877466,3.391224,2.633015,2.820816,1.763557,3.122672,...,2.063081,2.895849,2.816939,2.777088,2.847893,2.694402,0.594174,0.842804,2.903544,-0.337270
so**alweno,3.460342,3.142547,3.360626,3.442802,3.199069,3.500583,2.874763,3.060657,2.172545,3.310710,...,2.253330,3.246030,3.221896,3.201083,3.163364,2.896544,0.300162,1.148616,3.265018,-0.481683
so**amfe,3.139353,2.656372,2.753029,2.676407,2.859331,2.677628,2.401513,2.766397,1.775304,2.665514,...,1.813655,2.662253,2.514495,2.574309,2.403706,2.539436,0.428949,0.870799,2.752588,-0.382495
so**amigo,2.632208,2.689503,2.659193,2.674456,2.702948,2.645199,2.176779,2.662469,1.745654,2.679369,...,1.946732,2.791980,2.733227,2.666866,2.726003,2.722847,0.193847,0.975584,2.625132,-0.254479
so**an,3.243867,3.112239,3.241607,3.265894,3.140516,3.357900,2.873335,3.034054,2.015946,3.110123,...,2.351127,3.213535,3.041164,3.086964,3.047088,2.972160,0.495563,0.930624,3.289971,-0.412502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
so**ee73,2.349890,1.968865,2.014773,1.961411,2.115727,2.017993,1.785085,2.033780,1.314704,1.997346,...,1.332375,1.994304,1.844825,1.917999,1.767213,1.876145,0.400323,0.677821,2.053237,-0.283290
so**eehee,3.571594,3.751280,3.608289,3.716307,3.673593,3.859341,2.913623,3.572171,2.525820,3.624121,...,2.532253,3.740788,3.753568,3.618412,3.721286,3.673746,0.440099,1.416502,3.659881,-0.487443
so**eesook,3.260065,2.916952,3.332725,3.401030,3.175561,3.628372,2.947229,2.878636,2.129459,3.274928,...,2.230045,2.870618,2.867786,2.834489,2.905095,2.741848,0.718342,0.919116,3.229346,-0.496250


In [79]:
# 원본 행렬 확인
print(ratings_matrix2.shape)
ratings_matrix2.tail(100)

(50000, 3095)


title,-10KG 밀가루 단식,0.1%의 비밀,0~5세 뇌가 쑥쑥 자라는 놀이 육아,0~5세 말걸기 육아의 힘,1% 학습 코칭(99% 헛고생을 피하는),100 인생 그림책(Dear 그림책)(양장본 HardCover),1000개 숨은그림찾기: 바다 동물,1000개 숨은그림찾기: 아이스크림(찾아도 찾아도 끝판왕),1000개의 그림 1000가지 공감,100층짜리 집(양장본 HardCover),...,흔한남매의 THE 재즈소곡집 Very Easy,흔한남매의 흔한 호기심. 1,흔한남매의 흔한 호기심. 2,흔한남매의 흔한 호기심. 3,흔한남매의 흔한 호기심. 4,흔한남매의 흔한 호기심. 5,희망의 봄,히데코의 사적인 안주 교실,힐링 코드(개정증보판),힐튼호텔 옆 쪽방촌 이야기
id,,,,,,,,,,,,,,,,,,,,,
so**alcu8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
so**alweno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
so**amfe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
so**amigo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
so**an,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
so**ee73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
so**eehee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
so**eesook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 특정 사용자에게 아직 보지 않은 도서를 예측 평점 높은 순으로 추천해주기 

In [80]:
def get_unseen_movies(ratings_matrix2, id):
    # userId로 입력받은 사용자의 모든 서적정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 책명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix2.loc[id,:]
    
    # user_rating이 0보다 크면 기존에 읽은 서적이다. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    # 모든 책을 list 객체로 만듬. 
    movies_list = ratings_matrix2.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [81]:
def recomm_movie_by_userid(pred_df, id, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 서적 이름 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[id, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [82]:
# 사용자가 관람하지 않는 도서명 추출   
unseen_list = get_unseen_movies(ratings_matrix, "00**aith")

# 잠재 요인 협업 필터링으로 도서 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, "00**aith", unseen_list, top_n=20)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
일본어 무작정 따라하기,4.137795
7일의 기적 한국사능력검정시험 심화(1.2.3급)(2022)(큰별쌤 최태성의 별별한국사),4.127330
책읽는사자의 신앙의 참견,4.121820
튼이 이유식(세상 쉽고 맛있는),4.121059
일빵빵 기초영어: 동사 조동사 편(입에 달고 사는),4.109336
더 쉽고 더 맛있게 고단백 저탄수화물 다이어트 레시피,4.105902
지방 대사 켜는 스위치온 다이어트,4.104235
맛있게 살 빠지는 고단백 저탄수화물 다이어트 레시피,4.102057
엄마 나는 자라고 있어요,4.098503
